In [10]:
import numpy as np
from copy import copy, deepcopy
import queue
from collections import OrderedDict
import time
%run utilityFunc.ipynb
%run Node.ipynb
%run HeuristicFuntions.ipynb
%run CustomizedPriotiryQueue.ipynb

0
2
4


In [5]:
# read puzzle file
f = open("puzzle.txt", "r")
puzzle_list = [] # contains list of nparray for each puzzle
for line in f:
    # print(line) #test
    puzzle_nparr = np.array(list(line.split(" ")), dtype=int)
    dim1_len = int(len(puzzle_nparr)/4) #calculate length of first dimension
    puzzle_nparr = puzzle_nparr.reshape(dim1_len, 4) #reshape nparray to 2*4 multi-dimentional array
    puzzle_list.append(puzzle_nparr)
# print(puzzle_list)


puzzle_test = np.array([[3,0,1,4],[2, 6, 5, 7], [8, 9, 10, 11], [12,13,14,15]])
puzzle_dim_shape = puzzle_test.shape
sol = np.argwhere(puzzle_test == 15) # get index of specific element
print(sol)

[[3 3]]


In [3]:
# puzzle_test = regular_move(puzzle_dim_shape, sol, puzzle_test, "right")
# puzzle_test = wrapping_move(puzzle_dim_shape, sol, puzzle_test)
# puzzle_test = opposed_diagonally_move(puzzle_dim_shape, sol, puzzle_test)
# puzzle_test = adjacent_diagonally_move(puzzle_dim_shape, sol, puzzle_test)

# print(puzzle_test)

In [6]:
def get_successor(puzzle_dim_shape, elt_position, puzzle, cost):
    original_puzzle = deepcopy(puzzle)
    successor = []
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "up"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "down"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "left"), original_puzzle, cost+1))
    successor.append((regular_move(puzzle_dim_shape, elt_position, puzzle, "right"), original_puzzle, cost+1))
    successor.append((wrapping_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+2))
    successor.append((opposed_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))
    successor.append((adjacent_diagonally_move(puzzle_dim_shape, elt_position, puzzle), original_puzzle, cost+3))

    successor = [i for i in successor if i[0] is not None] # eliminate None puzzle node
    return successor


# return a list including all goal state
def generate_goal(puzzle):
    goal = []
    first_dim, second_dim = puzzle.shape
    puzzle_flatten = puzzle.flatten() # flatten to one dimentinal
    sorted_puzzle = np.sort(puzzle_flatten) # sort array
    sorted_puzzle = np.append(sorted_puzzle[1:], 0)

    goal1 = sorted_puzzle.reshape(first_dim, second_dim)
    goal.append(goal1)
    
    goal2 = np.full((first_dim, second_dim), 0)
    idx = 0
    for s in range(second_dim):
        for f in range(first_dim):
            goal2[f, s] = sorted_puzzle[idx]
            idx += 1
    goal.append(goal2)
    # print(goal)
    return goal


def check_goal(node, goal):
    current_node = node.get_current_puzzle()
    return (current_node == goal[0]).all() or (current_node == goal[1]).all()

# def check_goal(node, goal):
#     current_node = node.get_current_puzzle()
#     return (current_node == goal).all()

def is_exist_in_closedList(node, closed_list):
    isExist = False
    for curr_node in closed_list:
        isExist = isExist or curr_node.__eq__(node)
    return isExist


# get_successor(puzzle_dim_shape, sol, puzzle_test, 0)
# goal = generate_goal(puzzle_test)
# puzzle_test = deepcopy(goal[1])
# print(check_goal(puzzle_test, goal))



In [11]:
#Uniform Cost Algo
puzzle = puzzle_list[0]
puzzle_dim_shape = puzzle.shape
goal = generate_goal(puzzle)

def uniform_cost_algo(start_puzzle):
    start_time = time.time()
    total_cost = 0
    closed_list = set() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("uniform_cost") # list: [Node(node, parent_node, h_val, g_val), ...]
    open_queue.put(Node(start_puzzle, None, 0, total_cost))
    
    while open_queue:
        if time.time() - start_time >= 60.0:
            return None
        node = open_queue.get()
        print(node)
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        if not is_exist_in_closedList(node, closed_list):
            closed_list.add(node)
            
            if check_goal(node, goal):
                # ****************calculate the path
                return
            
            elt_position = np.argwhere(puzzle == 0)
            successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
            for node_tuple in successor:
                successor_node = Node(node_tuple[0], node_tuple[1], 0, node_tuple[2])
                if not is_exist_in_closedList(successor_node, closed_list):
                    open_queue.put(successor_node)
    return None

# test
# uniform_cost_algo(puzzle)


    

Node(current puzzle=[[1 2 3 4]
 [5 6 0 7]], parent_puzzle=None, h(node)=0, g(node)=0, f(node)=0)
Node(current puzzle=[[1 2 0 4]
 [5 6 3 7]], parent_puzzle=[[1 2 3 4]
 [5 6 0 7]], h(node)=0, g(node)=1, f(node)=1)
Node(current puzzle=[[1 2 3 4]
 [5 0 6 7]], parent_puzzle=[[1 2 3 4]
 [5 6 0 7]], h(node)=0, g(node)=1, f(node)=1)
Node(current puzzle=[[1 2 3 4]
 [5 6 7 0]], parent_puzzle=[[1 2 3 4]
 [5 6 0 7]], h(node)=0, g(node)=1, f(node)=1)


In [12]:
# greedy best first(GBFS) algo
def GBFS_algo(start_puzzle, goal_puzzle):
    total_cost = 0
    closed_list = set() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("GBFS")

    #calculate heuristic for start puzzle
    h = enhanced_Manhattan(start_puzzle, goal_puzzle) 
    open_queue.put(Node(start_puzzle, None, h, total_cost))
    
    while open_queue:
        node = open_queue.get()
        print(node)
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        # fix!!!!!!!
        if not is_exist_in_closedList(node, closed_list):
            closed_list.add(node)
            
            if check_goal(node, goal):
                return
            
            elt_position = np.argwhere(puzzle == 0)
            successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
            for node_tuple in successor:
                h = enhanced_Manhattan(node_tuple[0], goal_puzzle) 
                successor_node = Node(node_tuple[0], node_tuple[1], h, node_tuple[2])
                if not is_exist_in_closedList(successor_node, closed_list):
                    open_queue.put(successor_node)
    return None

# test
GBFS_algo(puzzle, goal)


Node(current puzzle=[[1 2 3 4]
 [5 6 0 7]], parent_puzzle=None, h(node)=1, g(node)=0, f(node)=1)
Node(current puzzle=[[1 2 3 4]
 [5 6 7 0]], parent_puzzle=[[1 2 3 4]
 [5 6 0 7]], h(node)=0, g(node)=1, f(node)=1)


In [13]:
puzzle = puzzle_list[0]
puzzle_dim_shape = puzzle.shape
goal = generate_goal(puzzle)

# A* algo
def aStarAlgo(start_puzzle, goal_puzzle):
    total_cost = 0
    closed_list = set() # elt: (Node(node, parent_node, h_val, g_val))
    open_queue = CustomizedPriotiryQueue("aStar")

    #calculate heuristic for start puzzle
    h = enhanced_Manhattan(start_puzzle, goal_puzzle) 
    open_queue.put(Node(start_puzzle, None, h, total_cost))
    
    while open_queue:
        node = open_queue.get()
        print(node)
        puzzle = node.get_current_puzzle()
        total_cost = node.get_g_val()

        if not is_exist_in_closedList(node, closed_list):
            closed_list.add(node)
            
            if check_goal(node, goal):
                return
            
            elt_position = np.argwhere(puzzle == 0)
            successor = get_successor(puzzle_dim_shape, elt_position, puzzle, total_cost)
            for node_tuple in successor:
                h = enhanced_Manhattan(node_tuple[0], goal_puzzle) 
                successor_node = Node(node_tuple[0], node_tuple[1], h, node_tuple[2])
                if not is_exist_in_closedList(successor_node, closed_list):
                    open_queue.put(successor_node)
    return None


aStarAlgo(puzzle, goal)

Node(current puzzle=[[1 2 3 4]
 [5 6 0 7]], parent_puzzle=None, h(node)=1, g(node)=0, f(node)=1)
Node(current puzzle=[[1 2 3 4]
 [5 6 7 0]], parent_puzzle=[[1 2 3 4]
 [5 6 0 7]], h(node)=0, g(node)=1, f(node)=1)
